In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting
import matplotlib as mpl
from matplotlib.collections import LineCollection

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,14,'FP2')
session.load()

In [ ]:
drivers = session.drivers

In [ ]:
tire = 'SOFT'
drivers = session.laps.pick_compounds(tire).Driver.unique()

In [ ]:
speed_dict = {}

for driver in drivers:
    dabb = session.get_driver(driver).Abbreviation
    color = plotting.get_driver_color(identifier=dabb,session=session)
    df = session.laps.pick_drivers(dabb).pick_quicklaps().pick_compounds(tire)
    min_speed, max_speed = [], []
    for i, lap in df.iterlaps():
        min_speed.append(lap.get_car_data().Speed.min())
        max_speed.append(lap.get_car_data().Speed.max())   
    speed_dict[session.get_driver(dabb).Abbreviation] = [max(min_speed), max(max_speed), color]

In [ ]:
speed_df = pd.DataFrame(speed_dict).T.set_axis(['min_speed','max_speed','color'],axis=1)

In [ ]:
# --- 1️⃣ Define ceilings (fastest driver speeds) ---
min_speed_ceiling = speed_df['min_speed'].max()
max_speed_ceiling = speed_df['max_speed'].max()

# --- 2️⃣ Calculate negative gaps (driver deficit) ---
speed_df['min_gap'] = speed_df['min_speed'] - min_speed_ceiling  # ≤ 0
speed_df['max_gap'] = speed_df['max_speed'] - max_speed_ceiling  # ≤ 0

# --- 3️⃣ Sort dataframe based on gaps (best performer first) ---
speed_df_sorted = speed_df.sort_values(by='min_gap', ascending=False)

# ----4️⃣ Sort again for max_gap (might differ from min_gap order) ----
speed_df_sorted2 = speed_df.sort_values(by='max_gap', ascending=False)

# Top driver in slow speeds
best_driver = speed_df_sorted.index[0]
best_speed = speed_df_sorted['min_speed'].iloc[0]
best_color = speed_df_sorted['color'].iloc[0]

# Top driver in straights
best_driver2 = speed_df_sorted2.index[0]
best_speed2 = speed_df_sorted2['max_speed'].iloc[0]
best_color2 = speed_df_sorted2['color'].iloc[0]

In [ ]:
# --- Plotting ---

setup_plot(xyticksize=20,axeslabel=20)

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# ---- Plot 1: Slow corner speed gaps ----
axes[0].bar(speed_df_sorted.index,
            speed_df_sorted['min_gap'],
            color=speed_df_sorted['color'])
axes[0].set_ylabel('Δ Speed (km/h)')
axes[0].set_title('Slow Corner Speed Deficit (compared to Best)')
axes[0].axhline(0, color='black', linewidth=10)

# ---- Plot 2: Straight-line speed gaps ----
axes[1].bar(speed_df_sorted2.index,
            speed_df_sorted2['max_gap'],
            color=speed_df_sorted2['color'])
axes[1].set_ylabel('Δ Speed (km/h)')
axes[1].set_title('Straight-line Speed Deficit (compared to Best)')
axes[1].axhline(0, color='black', linewidth=10)

axes[0].set_xlim(-0.5, len(speed_df_sorted) - 0.5)
axes[1].set_xlim(-0.5, len(speed_df_sorted2) - 0.5)

axes[0].text(
    x=0, y=-0.8,                           # position (x = bar index, y = just below axhline)
    s=f"{best_speed:.0f}",           # text to display
    color=best_color, fontsize=20,
    ha='center', va='bottom', fontweight='bold'
)

axes[1].text(
    x=0, y=-1.1,                           # position (x = bar index, y = just below axhline)
    s=f"{best_speed2:.0f}",           # text to display
    color=best_color2, fontsize=20,
    ha='center', va='bottom', fontweight='bold'
)

plt.tight_layout()
plt.show()

In [ ]:
fig.savefig('speed_analysis.png',dpi=300, bbox_inches='tight')

In [ ]:
lap = session.laps.pick_fastest()

In [ ]:
x = lap.telemetry['X']
y = lap.telemetry['Y']
color = lap.telemetry['Speed']

In [ ]:
import numpy as np

theta = np.deg2rad(39.5)  # Convert 45° to radians
x_rot = x * np.cos(theta) - y * np.sin(theta)
y_rot = x * np.sin(theta) + y * np.cos(theta)

In [ ]:
points = np.array([x_rot, y_rot]).T.reshape(-1,1,2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
fig.set_facecolor('#000000')
ax.axis('off')
ax.plot(x_rot, y_rot,
        color='#000000', linestyle='-', linewidth=16, zorder=0)

norm = plt.Normalize(color.min(), color.max())
colormap = mpl.cm.plasma
lc = LineCollection(segments, cmap=colormap, norm=norm,
                    linestyle='-', linewidth=10)

lc.set_array(color)

line = ax.add_collection(lc)

cbaxes = fig.add_axes([0.38, 0.5, 0.5, 0.05])
normlegend = mpl.colors.Normalize(vmin=color.min(), vmax=color.max())
legend = mpl.colorbar.ColorbarBase(cbaxes, norm=normlegend, cmap=colormap,
                                   orientation="horizontal")

for label in legend.ax.get_xticklabels():
    label.set_fontsize(16)
    label.set_fontweight('bold')
    label.set_color('w')

legend.ax.set_title('Speed (kmph)',color='w',fontdict=dict(fontsize=18,fontweight='bold'))
plt.tight_layout()
plt.show()